# FastAPI with Celery and Redis

In [1]:
import requests

In [2]:
response = requests.get('http://localhost:8000/add')
response = response.json()
task_id = response['task_id']
task_id

'd5cd09f5-0346-4803-9dfc-da2fa0adb182'

In [3]:
response = requests.get(f'http://localhost:8000/task/status/{task_id}')
print(response.json())

{'task_id': 'd5cd09f5-0346-4803-9dfc-da2fa0adb182', 'status': 'PENDING'}


In [6]:
response = requests.get(f'http://localhost:8000/task/result/{task_id}')
print(response.json())

{'status': 'completed', 'result': 10}


In [7]:
response = requests.get('http://localhost:8000/group/add')
response = response.json()
task_id = response['task_id']
task_id

'1edf3fa6-09cb-4fe8-9b1d-59de1de0547f'

In [8]:
response = requests.get(f'http://localhost:8000/group/status/{task_id}')
print(response.json())

{'successful': False}


In [12]:
response = requests.get(f'http://localhost:8000/group/result/{task_id}')
print(response.json())

{'status': 'completed', 'result': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]}


# Celery Group

In [13]:
from package.workers.worker import add
from package.workers.controller import celery_app
from celery.result import AsyncResult, GroupResult
from celery import group, chord, chain

In [14]:
jobs = group(add.s(i, i) for i in range(2))
result = jobs.apply_async()
result.save()
group_id = result.id

In [15]:
# results = GroupResult.restore(task_id, app=celery_app)
restored = GroupResult.restore(group_id, app=celery_app)

if restored.ready():
    print("All subtasks completed.")

All subtasks completed.


In [16]:
if restored.successful():
    _ = restored.get(timeout=5)
    print(_)

[0, 2]


In [17]:
restored.get(timeout=2)

[0, 2]